# Table of Contents
 <p>

In [1]:
import numpy as np
import pandas as pd
import sys
import time
import pickle
import glob
import regex as re
!pip3 install many_stop_words
from many_stop_words import get_stop_words
# stop_words = list(get_stop_words('en'))

from nltk.tokenize import RegexpTokenizer
import nltk
lemma = nltk.wordnet.WordNetLemmatizer()
from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english')))

In [2]:
# obtain df
files = glob.glob('/Users/jooppascha/Desktop/panda_pickles/*.p')
dfs = []
for f in files:
    dfs.append(pd.read_pickle(f))
df = pd.concat(dfs)

In [3]:
# obtain dataframe statistics
print('unique_rows: ', len(df))
df.head()

unique_rows:  4450000


,uri,subject,content,bestanswer,cat,maincat,subcat,document_type,language
0,1426578,whats your use now?,,"who is using you and why , you must answer tha...",Adolescent,Pregnancy & Parenting,Pregnancy & Parenting,wisdom,en-us
1,2520968,How to make a women have sex with you?,,In your case it would require a lot of alcohol...,Other - Family & Relationships,Family & Relationships,Family & Relationships,wisdom,en-us
2,795241,hey how to have an account with google?,................i mean i need an gmail account,"hey thats why am there, so sweet kiram just ma...",My Yahoo!,Yahoo! Products,Yahoo! Products,wisdom,en-us
3,2404030,What do you think?,"What's your opinion of this picture, FYI it is...","looks cool, strange colours though.",Astronomy & Space,Science & Mathematics,Science & Mathematics,wisdom,en-us
4,2040432,Is it possible for it to last and does long di...,ME and my boyfriend have been talking for abou...,Overseas Relationships are really hard to keep...,Other - Family & Relationships,Family & Relationships,Family & Relationships,wisdom,en-us


In [4]:
# selection criteria
min_character_len = 3
is_long_enough = (df.subject.str.len() >= min_character_len) & (df.bestanswer.str.len() >= min_character_len) 
has_no_content = (df.content.str.len() == 0) | (df.content == df.subject)
df = df[is_long_enough & has_no_content]

In [5]:
# smiley regex
eyes, noses, mouths = r":;8BX=", r"-~'^", r")(/\|DP"
pattern1 = "[%s][%s]?[%s]" % tuple(map(re.escape, [eyes, noses, mouths]))

smileys = """:-) :) :o) :] :3 :c) :> =] 8) =) :} :^) 
     :D 8-D 8D x-D xD X-D XD =-D =D =-3 =3 B^D""".split()
pattern2 = "|".join(map(re.escape, smileys))

In [6]:
tokenizer = RegexpTokenizer(r'\w+')
# reg_nums = re.compile(r"[+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?")

def pre_processing(string):
    # replace email adresses
    match = re.findall(r'[\w\.-]+@[\w\.-]+', string)
    for i in match:
        string = string.replace(i, ' email ')
    
    # replace any url
    string = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' link ', string)

    # remove tab characters
    string = string.strip()
    
    # replace smiley 
    string = re.sub(pattern1, 'smiley', string)
    string = re.sub(pattern2, 'smiley', string)
    
    # replace price
    string = re.sub(r'(USD|EUR|€|\$)\s?(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2}))|(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)\s?(USD|EUR|€|\$)', ' price ', string, flags=re.MULTILINE)
    
    # to lowercase
    string = string.lower()

    # replace year
    string = re.sub('[0-9][0-9][0-9][0-9]', ' year ', string, flags=re.MULTILINE)
    
    # match numbers
    string = re.sub(r"[+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?", ' number ', string, flags=re.MULTILINE)
    
    # remove non-words
    tokens = tokenizer.tokenize(string) 
    filtered_words = [t for t in tokens if t not in stop_words]

    string = ' '.join([lemma.lemmatize(t) for t in filtered_words])
    
    string = string.strip()
    return string

bestanswer = df.bestanswer.values
subject = df.subject.values

bestanswer_preprocessed = bestanswer.copy()
subject_preprocessed = subject.copy()

total_string = len(subject_preprocessed)
for i in range(len(bestanswer)):
    bestanswer_preprocessed[i] = pre_processing(bestanswer[i])
    subject_preprocessed[i] = pre_processing(subject[i])
    
    if i % 1000 == 0:
        sys.stdout.write("\r" + str((i/total_string)*100))

99.9615696201992763

In [7]:
df['bestanswer_preprocessed'] = bestanswer_preprocessed
df['subject_preprocessed'] = subject_preprocessed

In [9]:
min_character_len = 3
is_long_enough = (df.bestanswer_preprocessed.str.len() >= min_character_len) & (df.subject_preprocessed.str.len() >= min_character_len) 
df = df[is_long_enough]

In [10]:
df.to_pickle('preprocessing.p')

In [12]:
df.head()

,uri,subject,content,bestanswer,cat,maincat,subcat,document_type,language,bestanswer_preprocessed,subject_preprocessed
0,1426578,whats your use now?,,"who is using you and why , you must answer tha...",Adolescent,Pregnancy & Parenting,Pregnancy & Parenting,wisdom,en-us,using must answer question good good secretary...,whats use
1,2520968,How to make a women have sex with you?,,In your case it would require a lot of alcohol...,Other - Family & Relationships,Family & Relationships,Family & Relationships,wisdom,en-us,case would require lot alcohol rope,make woman sex
6,4339568,how do you set up mics on Logic Pro?,,I dnt knw but when you find out please tell me...,Other - Entertainment,Entertainment & Music,Entertainment & Music,wisdom,en-us,dnt knw find please tell lol im going thru pro...,set mics logic pro
9,2395756,MYSPACE HELP~Does anyone know where to find th...,,Just be very careful.Alot of stalkers and weir...,Computer Networking,Computers & Internet,Computers & Internet,wisdom,en-us,careful alot stalker weirdo scan myspace blog ...,myspace help anyone know find little questiona...
11,278601,How much of a left wing liberal imbecile do y...,,People here are dancing around this question.....,Other - Politics & Government,Politics & Government,Politics & Government,wisdom,en-us,people dancing around question vast majority l...,much left wing liberal imbecile believe number...
